In [ ]:
#******************Got 0.8855
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
import pandas as pd

def cost(x,y,m_arr):
    M = len(x)
    total_cost = 0
    for i in range(M):
        total_cost += (1/M)*((y[i] - (m_arr*x[i]).sum()) ** 2)
    return total_cost

def step_gradient(x,y,alpha,m_arr):
    M = len(x)
    N = len(x[0])
    m_slope = np.zeros(len(x[0]))
    for i in range(M):
        value = y[i]
        for j in range(N):
            value = (value - (m_arr[j]*x[i][j]).sum())
        for j in range(N):
            m_slope[j] +=(-2/M)*value*x[i][j]
    #print(m_slope)
    m_arr = m_arr - alpha*m_slope
    return m_arr

def gd(x,y,alpha,num_iterations):
    m_arr = np.zeros(len(x[0]))
    costs = []
    for i in range(num_iterations):
        costs.append(cost(x, y, m_arr)) 
        m_arr = step_gradient(x,y,alpha,m_arr)
        #print(m_arr)
    return m_arr,costs
    
def run(x,y):
    learning_rate = 0.1
    num_iterations = 200   
    m_arr,costs = gd(x,y,learning_rate,num_iterations)    
    plt.plot(costs)
    plt.show()
    return m_arr,costs

def predict(x,m_arr):
    summ = np.zeros(shape=len(x))
    summ = (m_arr*x)
    return summ
    
def score(Y_truth,Y_pred):
    u = ((Y_truth - Y_pred) ** 2).sum()
    v = ((Y_truth - Y_truth.mean()) ** 2).sum()
    return 1 - u/v

def cost(x,y,m_arr):
    M = len(x)
    total_cost = 0
    for i in range(M):
        total_cost += (1/M)*((y[i] - (m_arr*x[i]).sum()) ** 2)
    print(total_cost)
    return total_cost


df = pd.read_csv('train.csv', sep='\s*,\s*', header=0, encoding='ascii', engine='python')
target = df['EP']
x_df = df.drop(['EP'],axis=1)
new_df = df.drop(['EP'],axis=1)
for i in range(len(x_df.columns)):
    for j in range(i,len(x_df.columns)):
        name = x_df.columns[i]+"_"+x_df.columns[j]
        new_df[name] = x_df[x_df.columns[i]] * x_df[x_df.columns[j]]
df = new_df
df['EP'] = target

data = df.values
#data = data[:,[0,1,2,4,5,6,7,8,9,11,14]]
X_train = np.array(data)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
Y_train = np.array(data[:,-1])
X_train[:,-1] = 1
'''pca = PCA(n_components = 3) 
pca.fit(X_train) 
X_train = pca.transform(X_train)'''


m_arr,costs = run(X_train,Y_train)
Y_train_pred = predict(X_train,m_arr)
Y_train_modified_pred = np.zeros(len(Y_train_pred))
for i in range(len(Y_train_pred)):
    Y_train_modified_pred[i] = Y_train_pred[i].sum()
print("Training Score : ",score(Y_train,Y_train_modified_pred))




df = pd.read_csv('test.csv', sep='\s*,\s*', header=None, encoding='ascii', engine='python')
one_matrix = [1 for i in range(len(df))]
data = df.values
X_test = np.array(data)
df=pd.DataFrame(X_test) 
columns=["# T","V","AP","RH"]
df.columns = columns
n=len(df.columns)
for i in range(n):
    for j in range(i,n):
        r=df.columns[i]+'_'+df.columns[j]
        df[r]=df[df.columns[i]]*df[df.columns[j]]
df[15]=1

data = df.values
#data = data[:,[0,1,2,4,5,6,7,8,9,11,14]]
X_test = np.array(data)
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
X_test[:,-1] = 1

Y_test_pred = predict(X_test,m_arr)
Y_test_modified_pred = np.zeros(len(Y_test_pred))
for i in range(len(Y_test_pred)):
    Y_test_modified_pred[i] = Y_test_pred[i].sum()
    
df = pd.DataFrame()
list1 = []
for i in Y_test_modified_pred:
    k = "{0:.4f}".format(round(i,4))
    list1.append(k)
df = df.append(list1)
df.to_csv("plant_predictions.csv",index=None,header=None)